In [ ]:
import Pkg
Pkg.activate(".")

pkgs = [
"Revise",
"DataFrames",
"ProgressMeter",
"Statistics"
# "Iterators"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

import Mycelia

In [ ]:
uniref_dbs = [
    "UniRef100",
    "UniRef90",
    "UniRef50"
]

data_dir = joinpath(dirname(pwd()), "data")
SRR_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))

In [ ]:
# lca.tsv - how to interpret?
# contig
# classification taxon id
# rank
# name
# (1) a single taxonomy numeric identifier
# (2) a taxonomic rank column
# (3) taxonomic name column
# fragments retained
# fragments taxonomically assigned
# fragments in agreement with the contig label (i.e. same taxid or have it as an ancestor)
# the support received -log(E-value)

# report - this is tree-based relative frequencies - start here and make krona plots from these!

# tophit_aln - this seems to be a blast report, but it's not helpful because there aren't any fasta sequences to refer to
# (1,2) identifiers for query and target sequences/profiles, (3) sequence identity, (4) alignment length, (5) number of mismatches, (6) number of gap openings, (7-8, 9-10) domain start and end-position in query and in target, (11) E-value, and (12) bit score.
# query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits

# tophit_report
# (1) Target identifier 
# (2) Number of sequences aligning to target
# (3) Unique coverage of target uniqueAlignedResidues / targetLength
# (4) Target coverage alignedResidues / targetLength
# (5) Average sequence identity
# (6) Taxonomical information identifier, species, lineage 

In [ ]:
viral_frequency_table = DataFrames.DataFrame(
    sample_identifier = String[],
    classification_method = String[],
    percent_viral_contigs = Float64[]
)

ProgressMeter.@showprogress for SRR_path in SRR_paths
    mmseqs_dir = joinpath(SRR_path, "mmseqs_easy_taxonomy")
    for db in uniref_dbs
        kraken_report = "$(mmseqs_dir)/final.contigs.fastg.gfa.fna.mmseqs_easy_taxonomy.$(db)_report"
        # add something to determine everything below viruses with phylokit or whatever it's called
        viral_lines = collect(Iterators.filter(x -> occursin(r"virus"i, x) && occursin(r"10239"i, x), eachline(kraken_report)))
        if length(viral_lines) == 0
            percent_viral_contigs = 0.0
        elseif length(viral_lines) == 1
            viral_line_table = Mycelia.read_kraken_report(IOBuffer(join(viral_lines, '\n')))
            percent_viral_contigs = viral_line_table[1, "percentage_of_fragments_at_or_below_taxon"]
        else
            display(viral_lines)
            error()
        end
        row = (
            sample_identifier = basename(SRR_path),
            classification_method = db,
            percent_viral_contigs = percent_viral_contigs
        )
        push!(viral_frequency_table, row)
    end
end
viral_frequency_table

### summary_table = DataFrames.combine(DataFrames.groupby(viral_frequency_table, "sample_identifier"), "percent_viral_contigs" => Statistics.median)
sort(summary_table, "percent_viral_contigs_median", rev=true)

In [ ]:
SRR_path = first(SRR_paths)

In [ ]:
# make joint_contig_info reports

In [ ]:
readdir(SRR_path)

In [ ]:
# "genomad"
readdir(joinpath(SRR_path, "genomad", "final.contigs.fastg.gfa_summary"))
# "final.contigs.fastg.gfa_virus_genes.tsv"
# "final.contigs.fastg.gfa_virus_proteins.faa"

# "final.contigs.fastg.gfa_virus.fna"
# "final.contigs.fastg.gfa_virus_summary.tsv"

In [ ]:
# hit vs no hit
# "blastn"

In [ ]:
# "virsorter"
# # final-viral-score.tsv - 1 line per viral contig
# # final-viral-combined.fa - viral fasta records

In [ ]:
readdir(SRR_path)

In [ ]:
summary_table = DataFrames.combine(DataFrames.groupby(viral_frequency_table, "sample_identifier"), "percent_viral_contigs" => Statistics.median)

In [ ]:

for SRR_path in SRR_paths
    SRR = basename(SRR_path)
    assembled_fasta = joinpath(SRR_path, "megahit", "final.contigs.fastg.gfa.fna")
    out_dir = mkpath(joinpath(SRR_path, "mmseqs_easy_taxonomy"))
    outfile_base = joinpath(out_dir, basename(assembled_fasta) * ".mmseqs_easy_taxonomy." * basename(target_database))
    outfiles = [outfile_base * ext for ext in ["_lca.tsv", "_report", "_tophit_aln", "_tophit_report"]]
    if !all(isfile, outfiles)
        println("need to run $target_database for $SRR")
        # run(`sbatch mmseqs.sh $(assembled_fasta) $(target_database) $(outfile_base) $(joinpath(out_dir, "tmp"))`)
        # println(`sbatch mmseqs.sh $(assembled_fasta) $(target_database) $(outfile_base) $(joinpath(out_dir, "tmp"))`)
        # 73 minutes UniRef100
        # 35 minutes UniRef90
        # 10 minutes UniRef50
        # println("conda run --no-capture-output -n mmseqs2 mmseqs easy-taxonomy $(assembled_fasta) $(target_database) $(outfile_base) $(joinpath(out_dir, "tmp"))")
        # Mycelia.run_mmseqs_easy_taxonomy(out_dir=SRR_path, query_fasta=assembled_fasta, target_database=target_database, outfile=SRR)
    else
        println("all outfiles already exists for $target_database and $SRR...")
    end
end

In [ ]:
# python $HOME/workspace/Mycelia/projects/viral-pangenome-discovery/notebooks/kreport2krona.py -r $HOME/workspace/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399459/mmseqs_easy_taxonomy_default/SRR6399459.mmseqs_easy_taxonomy.UniRef100.txt_report -o $HOME/workspace/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399459/mmseqs_easy_taxonomy_default/SRR6399459.mmseqs_easy_taxonomy.UniRef100.txt_report.krona
# mamba install -c bioconda krona
# ktUpdateTaxonomy.sh
# ktImportText $HOME/workspace/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399459/mmseqs_easy_taxonomy_default/SRR6399459.mmseqs_easy_taxonomy.UniRef100.txt_report.krona -o $HOME/workspace/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399459/mmseqs_easy_taxonomy_default/SRR6399459.mmseqs_easy_taxonomy.UniRef100.txt_report.krona.html
# if !isfile(krona_file)
#     run(`python kreport2krona.py -r $(report) -o $(krona_file)`)
# end
# if !isfile(krona_html)
#     run(`ktImportText $(krona_file) -o $(krona_html)`)
# end